# Predictive Variables

In [ ]:
import os

import numpy as np
import xarray as xr
import rioxarray

import pandas as pd
import geopandas as gpd
from geocube.api.core import make_geocube

import matplotlib.pyplot as plt
import matplotlib as mpl

import cartopy.io.shapereader as shapereader

params = {'legend.fontsize': 'x-large',
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
plt.rcParams.update(params)

import shutil
import time
import sys
import glob
import subprocess

In [ ]:
data_path = os.path.join(os.path.dirname(os.getcwd()), 'data')
docs_path = os.path.join(os.path.dirname(os.getcwd()), 'docs')
figs_path = os.path.join(os.path.dirname(os.getcwd()), 'figs')
out_path = os.path.join(os.path.dirname(os.getcwd()), 'out', specie)
input_path = os.path.join(out_path, 'input', 'train') if training else os.path.join(out_path, 'input', 'test')
dirtmp = os.path.join(input_path,'tmp') # temporary directory
os.makedirs(dirtmp, exist_ok=True)

In [ ]:
regions = {
    'east-asia': ['China', 'Taiwan', 'Japan', 'North Korea', 'South Korea'],
    'indo': ["Indonesia",'Malaysia','Singapore','Brunei','East Timor'],
    'sea': ['Myanmar', 'Cambodia', 'Laos', 'Philippines', 'Thailand', 'Vietnam'],
    'south-east-asia': ['Brunei', 'Myanmar', 'Cambodia', 'East Timor', 'Indonesia', 'Laos', 'Malaysia', 'Philippines', 'Singapore', 'Thailand', 'Vietnam'],
    'australia': ['Australia'],
    'australasia' : ['Australia', 'New Zealand'],
    'india-sri-lanka' : ['Sri Lanka'],
    'all' : ['Australia','France','Italy','Portugal','South Africa','United States of America','Madagascar','Spain','Greece','Cyprus','Mexico','Kenya','Algeria','Israel','Egypt','Ethiopia','Ghana','Malawi','Mauritius','Morocco','Mozambique','Rwanda','Sierra Leone','United Republic of Tanzania','Tunisia','Uganda','Zimbabwe','China','India','Iran','Iraq','Jordan','Sri Lanka','Syria','Taiwan','Turkey','Malta','Montenegro','United Kingdom','Argentina','Brazil','Chile','Paraguay','Uruguay']
}

In [ ]:
# Load shapefiles of countries
gdf_countries = gpd.read_file(os.path.join(input_path, '%s.shp' %region))
shapes = [shape for shape in gdf_countries.geometry]

## 1. Elevation

https://www.ncei.noaa.gov/products/etopo-global-relief-model


In [ ]:
# etopo = rioxarray.open_rasterio(os.path.join(data_path, 'topo', 'ETOPO_2022_v1_60s_N90W180_bed.tif'), masked=True)
# etopo_east_asia = etopo.rio.clip(geometries=shapes)
# etopo_east_asia.rio.to_raster(os.path.join('output', 'rasters', 'etopo_east_asia.tif'))

In [ ]:
elev_worldclim = os.path.join(data_path, 'worldclim', 'wc2.1_30s_elev.tif')
srtm = rioxarray.open_rasterio(elev_worldclim, masked=True)
srtm_region = srtm.rio.clip(geometries=shapes)
# srtm_east_asia[np.where(srtm_east_asia < 0)].mask = True
srtm_region.name = 'elevation'
srtm_region.rio.to_raster(os.path.join(input_path, "tmp", 'srtm_%s.tif' %region))

In [ ]:
# 0.21875*8000
vmax = 0.85*srtm_region.max()
vmin = np.round((vmax - (1.21875 * vmax)))

fig, ax = plt.subplots(figsize=(18,6))
gdf_countries.plot(ax=ax, facecolor='lightgray', edgecolor='k')
# etopo_east_asia.plot(cmap=plt.cm.terrain, vmin=-2000, vmax=8000)
srtm_region.plot(ax=ax, cmap=plt.cm.gist_earth, vmin=vmin, vmax=vmax)
# ax.set_title('Elevation')

In [ ]:
if savefig:
    fig.savefig(os.path.join(figs_path, '03_wordclim_elevation_%s.png' %region), transparent=True)

## 2. Normalized Difference Vegetation Index (NDVI)
https://land.copernicus.eu/global/products/ndvi

In [ ]:
# https://land.copernicus.vgt.vito.be/manifest/ndvi_stats_all/manifest_cgls_ndvi_stats_all_latest.txt
# ndvi_nc_file = os.path.join(data_path, 'ndvi', 'c_gls_NDVI-STS_2015-2019-0611_GLOBE_PROBAV_V3.0.1.nc')
ndvi_nc_file = os.path.join(data_path, 'ndvi', 'c_gls_NDVI-LTS_1999-2019-1221_GLOBE_VGT-PROBAV_V3.0.1.nc')
ndvi_nc = xr.open_dataset(ndvi_nc_file)

In [ ]:
ndvi_nc.rio.write_crs("epsg:4326", inplace=True)
ndvi_nc_region = ndvi_nc.rio.clip(geometries=shapes)

In [ ]:
ndvi_median_region = ndvi_nc_region['median']

ndvi_class_bins = [-np.inf, 0, 0.1, 0.25, 0.4, np.inf]
ndvi_class = np.digitize(ndvi_median_region, ndvi_class_bins)

# Apply the nodata mask to the newly classified NDVI data
ndvi_class = np.ma.masked_where(np.isnan(ndvi_median_region), ndvi_class)

ndvi_median_class_region = xr.DataArray(ndvi_class, 
                                        coords={'lat': ndvi_median_region.lat,'lon': ndvi_median_region.lon,'crs': ndvi_median_region.crs}, 
                                        dims=["lat", "lon"])
ndvi_median_class_region.name = 'ndvi_median_classed'
ndvi_median_class_region.rio.to_raster(os.path.join(input_path, 'ndvi-median-classed_%s.tif' %region))

print('ndvi_median saved')

In [ ]:
col_dict = {0: ['white', 'Unknown'],
            1: ['gray', 'No Vegetation'],
            2: ['y', 'Bare Area'],
            3: ['yellowgreen', 'Low Vegetation'],
            4: ['g', 'Moderate Vegetation'],
            5: ['darkgreen', 'High Vegetation']}

cmap = mpl.colors.ListedColormap([col_dict[x][0] for x in col_dict.keys()])
labels = [col_dict[x][1] for x in col_dict.keys()]
cmap.set_bad='k'

norm = mpl.colors.BoundaryNorm(np.arange(-0.5,6), cmap.N) 
fmt = mpl.ticker.FuncFormatter(lambda x, pos: labels[norm(x)])

In [ ]:
# fig, ax = plt.subplots(ncols=2, figsize=(18, 6))

# gdf_countries.plot(ax=ax[0], facecolor='lightgray', edgecolor='k')
# pcol = ndvi_median_region.plot(ax=ax[0], cmap=plt.cm.YlGn, vmin=0, vmax=1, add_colorbar=False)
# plt.colorbar(pcol, extend='both')
# ax[0].set_title('Normalized Difference Vegetation Index')

# gdf_countries.plot(ax=ax[1], facecolor='lightgray', edgecolor='k')
# pcol = ndvi_median_class_region.plot(ax=ax[1], cmap=cmap, norm=norm, add_colorbar=False)
# plt.colorbar(pcol, format=fmt, ticks=np.linspace(0,5,6))
# ax[1].set_title('NDVI subdivided in classes')

# # Auto adjust subplot to fit figure size
# plt.tight_layout()

In [ ]:
# if savefig:
#     fig.savefig(os.path.join(figs_path, '03_ndvi_%s.png' %region), transparent=True)

In [ ]:
# fig, ax = plt.subplots(figsize=(8,6))

# gdf_countries.plot(ax=ax, facecolor='lightgray', edgecolor='k')
# # pcol = ndvi_median_region.plot(ax=ax, cmap=plt.cm.YlGn, vmin=0, vmax=1, add_colorbar=False)
# # fig.colorbar(pcol, fraction=0.02)

# pcol = ndvi_median_class_region.plot(ax=ax, cmap=cmap, norm=norm, add_colorbar=False)
# fig.colorbar(pcol, format=fmt, ticks=np.linspace(0,5,6), fraction=0.02)

# ax.set_title('NDVI subdivided in classes')
# ax.axis('off')
# fig.savefig(os.path.join(docs_path, '03_ndvi_%s.png' %region), transparent=True, dpi=600)
